# 🧪 Smart Waste Classifier Project

This notebook guides you through building a basic AI model to classify waste into four categories: Plastic, Paper, Metal, and Organic. It includes data loading, model training, evaluation, and export steps.

In [1]:
# 📦 Step 1: Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import os
data_dir = 'C:/Users/damia/Desktop/Sustainable Living Labs SL2/waste_dataset'
print("Exists:", os.path.exists(data_dir))
print("Folders:", os.listdir(data_dir) if os.path.exists(data_dir) else "Path not found")

Exists: False
Folders: Path not found


In [4]:
# 📂 Step 2: Load and Preprocess Dataset
# Only accepts JPG and PNG images (no wepg and avif images)
data_dir = 'C:/Users/damia/Desktop/SL2/waste_dataset'
img_height, img_width = 150, 150
batch_size = 16

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 38 images belonging to 4 classes.
Found 8 images belonging to 4 classes.


In [5]:
# 🧠 Step 3: Build the Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [7]:
from PIL import Image
import os

dataset_dir = 'C:/Users/damia/Desktop/SL2/waste_dataset'

for folder in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, folder)
    if not os.path.isdir(folder_path):
        continue
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            with Image.open(file_path) as img:
                img.verify()  # Check if it's a valid image
        except Exception:
            print(f"❌ Removing: {file_path} (not a valid image)")
            os.remove(file_path)


❌ Removing: C:/Users/damia/Desktop/SL2/waste_dataset\metal\desktop.ini (not a valid image)
❌ Removing: C:/Users/damia/Desktop/SL2/waste_dataset\organic\desktop.ini (not a valid image)
❌ Removing: C:/Users/damia/Desktop/SL2/waste_dataset\paper\desktop.ini (not a valid image)
❌ Removing: C:/Users/damia/Desktop/SL2/waste_dataset\plastic\desktop.ini (not a valid image)


# 30 imgs - What level of accuracy are you going for? 

In [8]:
# 🔁 Step 4: Train the Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

Epoch 1/10
3/3 [==============================] - 2s 444ms/step - loss: 5.2165 - accuracy: 0.2105 - val_loss: 8.9289 - val_accuracy: 0.2500
Epoch 2/10
3/3 [==============================] - 1s 350ms/step - loss: 6.0350 - accuracy: 0.2632 - val_loss: 2.0711 - val_accuracy: 0.2500
Epoch 3/10
3/3 [==============================] - 1s 309ms/step - loss: 1.3838 - accuracy: 0.3684 - val_loss: 1.3567 - val_accuracy: 0.3750
Epoch 4/10
3/3 [==============================] - 1s 286ms/step - loss: 1.1369 - accuracy: 0.5789 - val_loss: 1.2853 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 1s 354ms/step - loss: 0.6197 - accuracy: 0.8684 - val_loss: 1.1066 - val_accuracy: 0.5000
Epoch 6/10
3/3 [==============================] - 1s 387ms/step - loss: 0.4380 - accuracy: 0.8421 - val_loss: 1.3312 - val_accuracy: 0.6250
Epoch 7/10
3/3 [==============================] - 1s 381ms/step - loss: 0.3636 - accuracy: 0.8684 - val_loss: 1.4886 - val_accuracy: 0.6250
Epoch 8/10
3/3 [====

In [9]:
# 📊 Step 5: Evaluate the Model
val_data.reset()
predictions = model.predict(val_data)
y_pred = np.argmax(predictions, axis=1)
y_true = val_data.classes

print("Classification Report:\n", classification_report(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

1/1 [==============================] - 0s 344ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       2.0
           2       0.00      0.00      0.00       2.0
           3       0.00      0.00      0.00       2.0

    accuracy                           0.00       8.0
   macro avg       0.00      0.00      0.00       8.0
weighted avg       0.00      0.00      0.00       8.0

Confusion Matrix:
 [[0 1 0 1]
 [1 0 0 1]
 [1 1 0 0]
 [0 1 1 0]]


In [10]:
# 💾 Step 6: Save the Model
model.save('waste_classifier_model.h5')